# Analysis

* Enter - edit
* M - switch to markdown
* Y - switch to code
* ctrl-enter - run cell
* A - add above
* B - add below

Shared functions for analysis

In [41]:
import re

import pandas as pd
import numpy as np

pd.set_option("display.max_rows", 100)

# constants
ALL_OUTPUT = "/home/dubs/dev/paap/data/output/calculate_modern_price_output.json"
PHOTOS_AND_PRINTS = "/home/dubs/dev/paap/data/output/photos_and_prints_only.json"

In [8]:
df_all = pd.read_json(ALL_OUTPUT, orient="records")

In [9]:
df_all

,id,sale_number,sale_url,sale_input_url,sale_year,sale_month,sale_date,sale_location,sale_location_int,sale_category,...,lot_estimate_low,lot_estimate_high,lot_description,lot_medium,lot_image_id,usd_equivalent,inflation_to_2020,lot_realized_price_usd,lot_estimate_low_usd,lot_estimate_high_usd
0,d9389924-38c7-4411-b042-345b2993e6e4,1756,https://www.christies.com/results?sc_lang=en&m...,https://www.christies.com/results?sc_lang=en&m...,2006,10,2006-10-03,new_york,43,asian_art,...,5000,7000,"Bolognese School, 16th Century",,812944f7-7607-45f9-ac1b-966b14cfaad8,1.0,1.278350,6136.079286,6391.749257,8948.448959
1,5e53cd6f-cb93-45a9-bd65-191fe5b336d6,1756,https://www.christies.com/results?sc_lang=en&m...,https://www.christies.com/results?sc_lang=en&m...,2006,10,2006-10-03,new_york,43,asian_art,...,5000,7000,After Correggio,,6a873c3b-1672-4996-b492-c04889326e2f,1.0,1.278350,5829.275322,6391.749257,8948.448959
2,8852dd94-6719-4522-8492-2da04e2fcd1b,1756,https://www.christies.com/results?sc_lang=en&m...,https://www.christies.com/results?sc_lang=en&m...,2006,10,2006-10-03,new_york,43,asian_art,...,1000,1500,"AFTER RAPHAEL, 20TH CENTURY",,ec5fab69-3aeb-4473-bb1a-61566c969c05,1.0,1.278350,153.401982,1278.349851,1917.524777
3,0da60937-40f3-41c3-8681-865bc981a04e,1756,https://www.christies.com/results?sc_lang=en&m...,https://www.christies.com/results?sc_lang=en&m...,2006,10,2006-10-03,new_york,43,asian_art,...,2000,3000,"After Andrea del Sarto, early 19th Century",,93366520-7112-47d9-bbff-f5e475b221a2,1.0,1.278350,2914.637661,2556.699703,3835.049554
4,eeba8512-4fd0-48d9-b212-5d6c84143d0e,1756,https://www.christies.com/results?sc_lang=en&m...,https://www.christies.com/results?sc_lang=en&m...,2006,10,2006-10-03,new_york,43,asian_art,...,5000,7000,Circle of Jan Miel (Beveren-Waes 1599-1664 Turin),,f2e1e70c-a4da-4422-b0bb-a63466da5b16,1.0,1.278350,4602.059465,6391.749257,8948.448959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259591,1a576346-ceec-4b96-8fb9-6207264c0703,18084,https://onlineonly.christies.com/s/matisse-pap...,https://www.christies.com/results?sc_lang=en&m...,2019,3,2019-03-01,online,115,photographs_and_prints,...,7000,10000,Henri Matisse (1869-1954)<br>Pot au fleurs<br>...,None,c03b6bfe-a7b6-4445-845a-60d26fcef8fb,1.0,1.014827,19028.002337,7103.787539,10148.267913
259592,c2f91534-cecc-4350-b54a-e6d955a7535d,18084,https://onlineonly.christies.com/s/matisse-pap...,https://www.christies.com/results?sc_lang=en&m...,2019,3,2019-03-01,online,115,photographs_and_prints,...,8000,12000,HENRI MATISSE (1869-1954)<br>Nu renversé au br...,None,ef6cadc9-8d1c-4929-b1ed-6f077b2307ee,1.0,1.014827,30444.803739,8118.614330,12177.921496
259593,82e9e786-12df-48e4-b2dc-26663573452e,18084,https://onlineonly.christies.com/s/matisse-pap...,https://www.christies.com/results?sc_lang=en&m...,2019,3,2019-03-01,online,115,photographs_and_prints,...,8000,12000,"HENRI MATISSE (1869-1954)<br>Nu à genoux, bras...",None,05f36344-3821-43a1-b841-0ebf843cec48,1.0,1.014827,19028.002337,8118.614330,12177.921496
259594,2804252d-35ec-4b59-93b7-e1b6ec9be955,18084,https://onlineonly.christies.com/s/matisse-pap...,https://www.christies.com/results?sc_lang=en&m...,2019,3,2019-03-01,online,115,photographs_and_prints,...,5000,7000,HENRI MATISSE (1869-1954)<br>Nu assis au colli...,None,0939e479-83dd-49e6-bda0-22e002fd4200,1.0,1.014827,22833.602804,5074.133956,7103.787539


In [15]:
df_all["sale_category"].value_counts()

fine_art                        162081
photographs_and_prints           51173
asian_art                        42187
islamic_and_eastern_european      4155
Name: sale_category, dtype: int64

In [35]:
# df_all["lot_artist"].value_counts()[:100]
df_all["lot_description"].value_counts()[:100].values

36177

One option would be to take the top x represented artists. You would then want
the model to be fairly adept at picking those artists. Would be interesting to
see which artworks were not selected by the model.

Another option would be to take a weighted random sample of the top n artists and
cluster to something mush smaller than that

Could also take all black and white prints and cluster on that. Might not have
enough data to do that well.

In [53]:
# Select the top 100 artists by number of instances in the dataset
MAKER_RE = re.compile(r"^(?P<maker>[\w\s\.,\-'']+)\s\([\s0-9bBb\-\.]+\)$")

def strip_date(x):
    match = re.search(MAKER_RE, x)
    if not match:
        return x
    return match.group("maker")

df_all["lot_description"] = df_all["lot_description"].apply(lambda x: x.lower().strip())
df_all = df_all[~df_all.lot_description.isin(["a", "a", "none" "", " ", "english school, 19th century", "after pablo picasso"])]
df_all["lot_description"] = df_all["lot_description"].apply(lambda x: x if x != "joan miró" else "joan miro")
df_all["lot_description"] = df_all["lot_description"].apply(strip_date)
top_100 = df_all["lot_description"].value_counts()[:100]
top_100

pablo picasso                   4832
andy warhol                     3120
joan miro                       1838
marc chagall                    1474
rembrandt harmensz. van rijn     912
david hockney                    911
roy lichtenstein                 880
henri matisse                    860
albrecht dürer                   710
francis newton souza             624
henry moore                      616
alexander calder                 559
robert rauschenberg              539
zhang daqian                     538
sam francis                      535
irving penn                      529
frank stella                     515
damien hirst                     493
jean dubuffet                    475
jasper johns                     472
victor vasarely                  464
tom wesselmann                   459
lucio fontana                    455
gerhard richter                  446
pierre-auguste renoir            437
zao wou-ki                       433
sol lewitt                       426
h